In [ ]:
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.graphs.hypergraph_methods import calculate_full_cost_hetero
from qiskit import transpile
from disqco.parti.FM.FM_methods import set_initial_partitions
from disqco.circuits.QAOA import QAOA_random
from qiskit.circuit.library import QFT, QuantumVolume
import pickle
from copy import deepcopy
from disqco.graphs.quantum_network import *
from disqco.parti.FM.multilevel_FM import MLFM_recursive_hetero
import networkx as nx
from copy import deepcopy
import numpy as np
from disqco.graphs.quantum_network import QuantumNetwork
import os
import json

num_qubits_list = [512]
network_types = ['linear']
num_partitions_and_coarsening_factors = [(32, 4), (64,4)]

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_COARENING_POST_PROCESSED_limit8_add.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

In [ ]:
from disqco.parti.FM.net_coarsened_FM import run_net_coarsened_FM
from disqco.graphs.quantum_network import linear_coupling, grid_coupling
import time
iterations = 20

for num_qubits in num_qubits_list:
    for network_type in network_types:
        for num_partitions_and_coarsening_factor in num_partitions_and_coarsening_factors:
            level_limit = 8  # Set the level limit for coarsening
            for iteration in range(iterations):
                num_partitions, coarsening_factor = num_partitions_and_coarsening_factor
                print(f"num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, coarsening_factor: {coarsening_factor}")
                # Check first whether the json file already contains results for this configuration
                existing_results = [entry for entry in detailed_results if entry['num_qubits'] == num_qubits and 
                                    entry['network_type'] == network_type and
                                    entry['num_partitions'] == num_partitions and
                                    entry['coarsening_factor'] == coarsening_factor]
                if existing_results:
                    if existing_results[0]['final_cost'] is not None:
                        print(f"Results already exist for num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, coarsening_factor: {coarsening_factor}. Skipping this configuration.")
                        continue
                
                seed = np.random.randint(0, 10000)
                
                circuit = cp_fraction(num_qubits=num_qubits,
                                        depth=num_qubits,
                                        fraction=0.5,
                                        seed=seed)
                
                circuit = transpile(circuit, basis_gates=['cp', 'u'])

                qpu_sizes = [int(np.ceil(num_qubits / num_partitions)) + 1 for i in range(num_partitions)]

                if network_type == 'linear':
                    coupling = linear_coupling(num_partitions)
                elif network_type == 'grid':
                    coupling = grid_coupling(num_partitions)

                initial_qpu_sizes = {i: qpu_sizes[i] for i in range(num_partitions)}
                initial_network = QuantumNetwork(qpu_sizes, coupling)

                network = deepcopy(initial_network)
                initial_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
                depth = initial_graph.depth
                graph_no_grouping = QuantumCircuitHyperGraph(circuit, group_gates=False, anti_diag=True)

                graph = deepcopy(initial_graph)
                assignment = set_initial_partitions(network, num_qubits, depth)

                initial_cost_no_grouping = calculate_full_cost_hetero(hypergraph=graph_no_grouping, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial cost with no grouping: ", initial_cost_no_grouping)

                initial_cost = calculate_full_cost_hetero(hypergraph=graph, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial Cost: ", initial_cost)

                try:
                    start_time = time.time()
                    cost, final_assignment = run_net_coarsened_FM(graph, network, l=coarsening_factor, multiprocessing=True, level_limit=level_limit, passes_per_level=10)
                    final_cost = calculate_full_cost_hetero(hypergraph=graph, assignment=final_assignment, num_partitions=num_partitions, costs = {}, network=network)
                    end_time = time.time()
                    print("Final Cost: ", final_cost)
                    print("Time taken: ", end_time - start_time)
                    results_entry = {
                        "num_qubits": num_qubits,
                        "network_type": network_type,
                        "num_partitions": num_partitions,
                        "coarsening_factor": coarsening_factor,
                        "level_limit": level_limit,
                        "initial_cost_no_grouping": initial_cost_no_grouping,
                        "initial_cost": initial_cost,
                        "final_cost": final_cost,
                        "iteration": iteration,
                        "time_taken": end_time - start_time,
                        "seed": seed
                    }
                    detailed_results.append(results_entry)


                except Exception as e:
                    raise e
                    print(f"Error occurred: {e}")
                    results_entry = {
                        "num_qubits": num_qubits,
                        "network_type": network_type,
                        "num_partitions": num_partitions,
                        "coarsening_factor": coarsening_factor,
                        "level_limit": level_limit,
                        "initial_cost_no_grouping": initial_cost_no_grouping,
                        "initial_cost": initial_cost,
                        "final_cost": None,
                        "iteration": iteration,
                        "time_taken": None,
                        "seed": seed
                    }
                    detailed_results.append(results_entry)
                    if level_limit > 1:
                        level_limit -= 1
                        
                        
                    

                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)


num_qubits: 512, network_type: linear, num_partitions: 64, coarsening_factor: 4
Number of layers: 514
Number of layers: 514
Initial cost with no grouping:  1246421
Initial Cost:  797866
Time to coarsen graph: 7.30 seconds
Time for multilevel FM: 384.51 seconds
Time to coarsen graph: 6.12 seconds
Time to coarsen graph: 8.19 seconds
Time to coarsen graph: 7.59 seconds
Time to coarsen graph: 6.13 seconds
Time for multilevel FM: 56.96 seconds
Time for multilevel FM: 52.82 seconds
Time for multilevel FM: 51.31 seconds
Time for multilevel FM: 55.35 seconds
Time to coarsen graph: 4.37 seconds
Time to coarsen graph: 3.57 seconds
Time to coarsen graph: 5.22 seconds
Time for multilevel FM: 13.44 seconds
Time to coarsen graph: 4.06 seconds
Error processing node (510, 432) with sub_node (36, 432): index 36 is out of bounds for axis 0 with size 36
Time for multilevel FM: 14.18 seconds
Time for multilevel FM: 11.27 seconds
Time to coarsen graph: 5.53 seconds
Time to coarsen graph: 5.94 seconds
Time 

IndexError: index 36 is out of bounds for axis 0 with size 36

In [ ]:
# Now set up same testing loop using standard MLFM-R
# Read through JSON file to find random seeds to generate each circuit
import time


from disqco.parti.FM.multilevel_FM import MLFM_recursive_hetero

# final_assignment_list, final_cost_list, _ = MLFM_recursive_hetero(graph,
#                                                                     assignment,
#                                                                     qpu_sizes,
#                                                                     limit=num_qubits,
#                                                                     network=network,
#                                                                     log=True,
#                                                                     stochastic=True,
#                                                                     costs=costs, 
#                                                                     level_limit=None)


with open(detailed_filename, "r") as f:
    detailed_results = json.load(f)


direct_partitioning_filename = "benchmark_results_COARSENING_POST_PROCESSED_direct_partitioning.json"
if os.path.exists(direct_partitioning_filename):
    with open(direct_partitioning_filename, "r") as f:
        direct_results = json.load(f)
else:
    direct_results = []

completed_set = set()

for result in direct_results:
    num_qubits = result['num_qubits']
    network_type = result['network_type']
    num_partitions = result['num_partitions']
    completed_set.add((num_qubits, network_type, num_partitions))

for result in detailed_results:

    num_qubits = result['num_qubits']
    network_type = result['network_type']
    num_partitions = result['num_partitions']
    level_limit = result['level_limit']
    initial_cost_no_grouping = result['initial_cost_no_grouping']
    initial_cost = result['initial_cost']
    final_cost = result['final_cost']
    iteration = result['iteration']
    time_taken = result['time_taken']
    seed = result['seed']
    if num_qubits is not 512 and network_type != 'grid' and num_partitions != 4:
        continue
    if (num_qubits, network_type, num_partitions) in completed_set:
        print(f"Skipping already completed configuration: num_qubits={num_qubits}, network_type={network_type}, num_partitions={num_partitions}")
        continue
    completed_set.add((num_qubits, network_type, num_partitions))
    
    if num_partitions <= 16:
        level_limit = 8
    else:
        results_entry = {
            "num_qubits": num_qubits,
            "network_type": network_type,
            "num_partitions": num_partitions,
            "level_limit": 1,
            "initial_cost_no_grouping": initial_cost_no_grouping,
            "initial_cost": initial_cost,
            "final_cost": initial_cost,  # No final cost for direct partitioning as too large
            "iteration": iteration,
            "time_taken": None,
            "seed": seed
        }

        direct_results.append(results_entry)
        with open(direct_partitioning_filename, "w") as f:
            json.dump(direct_results, f, indent=2)
        print(f"Skipping direct partitioning for num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions} as level_limit is 1")
        continue
    
    print(f"Direct partitioning for num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, level_limit: {level_limit}")
    circuit = cp_fraction(num_qubits=num_qubits,
                            depth=num_qubits,
                            fraction=0.5,
                            seed=seed)
    
    circuit = transpile(circuit, basis_gates=['cp', 'u'])

    qpu_sizes = [int(np.ceil(num_qubits / num_partitions)) + 1 for i in range(num_partitions)]

    if network_type == 'linear':
        coupling = linear_coupling(num_partitions)
    elif network_type == 'grid':
        coupling = grid_coupling(num_partitions)

    initial_qpu_sizes = {i: qpu_sizes[i] for i in range(num_partitions)}
    network = QuantumNetwork(qpu_sizes, coupling)


    graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)

    depth = graph.depth

    assignment = set_initial_partitions(network, num_qubits, depth)
    start_time = time.time()
    final_assignment_list, final_cost_list, _ = MLFM_recursive_hetero(graph,
                                                                    assignment,
                                                                    qpu_sizes,
                                                                    limit=num_qubits,
                                                                    network=network,
                                                                    log=True,
                                                                    stochastic=True,
                                                                    costs={},
                                                                    level_limit=level_limit)
    end_time = time.time()

    time_taken = end_time - start_time
    
    results_entry = {
        "num_qubits": num_qubits,
        "network_type": network_type,
        "num_partitions": num_partitions,
        "level_limit": level_limit,
        "initial_cost_no_grouping": initial_cost_no_grouping,
        "initial_cost": initial_cost,
        "final_cost": min(final_cost_list),
        "iteration": iteration,
        "time_taken": time_taken,
        "seed": seed}
    
    direct_results.append(results_entry)

    with open(direct_partitioning_filename, "w") as f:
        json.dump(direct_results, f, indent=2)


<>:50: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
<>:50: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
/var/folders/s5/7cd6cqh90fd7qlk1kl1bnr0w0000gp/T/ipykernel_38111/3993395770.py:50: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
  if num_qubits is not 512 and network_type != 'grid' and num_partitions != 4:


Skipping already completed configuration: num_qubits=128, network_type=linear, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=linear, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=linear, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=linear, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=linear, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=4
Skipping already completed configuration: num_qubits=128, network_type=grid, num_partitions=4
Skipping already completed configuration: num_qubi

KeyboardInterrupt: 